In [2]:
import sdmx # pip install sdmx1 # https://pypi.org/project/sdmx1/
istat = sdmx.Client("ISTAT")

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
istat = sdmx.Client('Istat')
key = dict(FREQ="A", ITTER107="IT+ITC+ITD+ITE+ITF+ITG", TIPO_AGGR ='B1G_B_W2_S1_R_PS', VAL = 'V' , EDI ='2023M12')
params = dict(startPeriod='2010', endPeriod='2022')
data_msg = istat.data('93_500', key=key, params=params)
data = data_msg.data[0]
data_df = data_msg.to_pandas()
data_df2 = sdmx.to_pandas(data,
        datetime={'dim': 'TIME_PERIOD',
                'freq': 'FREQ',
                'axis': 1})
data_final = data_df2.xs(
    level=("TIPO_AGGR", "VAL", "EDI","ADJUSTMENT", "BRANCA_ATTIV_REV2","DCCN_COICOP_COFOG","PRODOTTI1","TIPPREZ"), # 
    key=("B1G_B_W2_S1_R_PS", "V", "2023M12","N","Z","Z","Z","Z"))        # 
data_final

ValueError: Dimensions ['ITTER107', 'TIPO_AGGR', 'VAL', 'EDI'] not in DimensionDescriptor(annotations=[], id='DimensionDescriptor', uri=None, urn=None, components=[<Dimension FREQ>, <Dimension REF_AREA>, <Dimension DATA_TYPE_AGGR>, <Dimension BRKDW_INDUSTRY_NACE_REV2>, <Dimension NONFIN_ASSETS>, <Dimension EXPEND_PURPOSE_COICOPCOFOG>, <Dimension VALUATION>, <Dimension ADJUSTMENT>, <Dimension PRICE>, <Dimension EDITION>, <TimeDimension TIME_PERIOD>])

In [ ]:
istat = sdmx.Client('Istat')
key = dict(FREQ="A", ITTER107="IT", VAL = 'V' , EDI ='2023M12')
params = dict(startPeriod='2010', endPeriod='2023')
data_msg = istat.data('93_500', key=key, params=params)
data = data_msg.data[0]
data_df = data_msg.to_pandas()
data_df2 = sdmx.to_pandas(data,
        datetime={'dim': 'TIME_PERIOD',
                'freq': 'FREQ',
                'axis': 1})

data_final = data_df2.xs(
    level=("VAL", "EDI","BRANCA_ATTIV_REV2","ADJUSTMENT", "DCCN_COICOP_COFOG","PRODOTTI1","TIPPREZ"),
    key=("V", "2023M12","Z","N","Z","Z","Z"))        
data_final

In [ ]:
sdmx.to_pandas(data,datetime={'dim': 'TIME_PERIOD','freq': 'FREQ','axis': 1})

| Oggetto                             | Significato                                                                                          |
| ----------------------------------- | ---------------------------------------------------------------------------------------------------- |
| **Dataflow (DF)**                   | Il contenitore del dataset (es. `92_1225` = Investimenti pubblici)                                   |
| **DSD (Data Structure Definition)** | La struttura del dataset, cioè le **dimensioni**, **unità di misura**, **codici validi**             |
| **Dimensioni**                      | Gli assi lungo cui si articolano i dati: es. `TERRITORIO`, `SESSO`, `ANNO`, `FASCIA_ETÀ`             |
| **Codelist**                        | L’insieme dei valori validi per ciascuna dimensione (es. tutte le regioni italiane per `TERRITORIO`) |
| **Osservazioni**                    | I valori numerici veri e propri, definiti da una combinazione di dimensioni                          |

---

| Passo | Cosa fai                    | Strumento            |
| ----- | --------------------------- | -------------------- |
| 1     | Trovi il dataset            | `dataflow()`         |
| 2     | Ne capisci la struttura     | `get(datastructure)` |
| 3     | Esplori i codici            | `.codelist[...]`     |
| 4     | Costruisci la chiave filtro | dizionario Python    |
| 5     | Richiedi i dati             | `data()`             |
| 6     | Li analizzi                 | `to_pandas()`        |


## 92_505 sequenza dei conti

Il dataset 92_505 – Sequenza dei conti dell'ISTAT fa parte della Contabilità Nazionale e rappresenta la sequenza dei conti economici secondo il Sistema Europeo dei Conti (SEC). Questo dataset fornisce una visione dettagliata dei flussi economici tra i diversi settori istituzionali dell'economia italiana.

📊 Contenuto del dataset
Il dataset include informazioni su:

Conti di produzione: valore aggiunto generato dai settori produttivi.

Conti del reddito: distribuzione e redistribuzione del reddito tra settori.

Conti finanziari: transazioni finanziarie tra settori.

Conti patrimoniali: stock di attività e passività.
datiistat.it - l'Istat semplice

🧩 Dimensioni principali
Le principali dimensioni del dataset includono:

FREQ: Frequenza (annuale, trimestrale).

REF_AREA: Territorio (es. Italia, regioni).

SECTOR: Settore istituzionale (es. famiglie, imprese, amministrazioni pubbliche).

TRANSACTION: Tipo di transazione (es. salari, imposte, investimenti).

VALUATION: Valutazione (es. prezzi correnti, prezzi costanti).

UNIT: Unità di misura (es. milioni di euro).
SDMX Istat

🔍 Utilizzo del dataset
Questo dataset è fondamentale per:

Analizzare la distribuzione del reddito e della spesa tra i settori.

Valutare l'impatto delle politiche fiscali e monetarie.

Studiare le relazioni economiche tra i diversi attori dell'economia.

📥 Accesso ai dati
Puoi accedere al dataset attraverso i servizi web SDMX dell'ISTAT:

Endpoint SDMX: https://sdmx.istat.it/SDMXWS/rest

Dataflow ID: 92_505

Codice SDMX: DCCN_SQCN
SDMX Istat
+1
Istat
+1
datiistat.it - l'Istat semplice

In [23]:
import sdmx
istat = sdmx.Client("ISTAT")

# 1. Carica il dataflow
dataflow_id = '92_505'
# dataflow_id = 'DF_DCCN_TNA1_6'
flow_msg = istat.dataflow(dataflow_id)
dfd = flow_msg.dataflow[dataflow_id]

# 2. Carica la struttura
structure_id = dfd.structure.id
structure_msg = istat.get(resource_type="datastructure", resource_id=structure_id)

# 3. Estrai la struttura vera e propria
dsd = structure_msg.structure[0]

# 4. Elenco dimensioni in italiano
print("\n📏 Dimensioni:")
for dim in dsd.dimensions:
    concept = dim.concept_identity
    nome_it = concept.name.localizations.get('it', concept.id)
    print(f"- {dim.id}: {nome_it}")




📏 Dimensioni:
- FREQ: Frequenza
- REF_AREA: Territorio
- DATA_TYPE_AGGR: Aggregato
- VALUATION: Valutazione
- ADJUSTMENT: Correzione
- EDITION: Edizione
- TIME_PERIOD: Tempo


In [14]:
dfd

<DataflowDefinition IT1:92_505(1.0): Sequence of accounts>

In [15]:
dsd

<DataStructureDefinition IT1:DCCN_SQCN(1.0): Sequence of accounts>

## PIL regionale

In [ ]:
import requests
import xml.etree.ElementTree as ET

url = "https://esploradati.istat.it/SDMXWS/rest/datastructure/ISTAT/93_498_DF_DCCN_PILT_1"
r = requests.get(url)
r.raise_for_status()

root = ET.fromstring(r.content)
ns = {'str': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure'}

dims = root.findall('.//str:Dimension', ns)
print("✅ Dimensioni attese:")
for dim in dims:
    print("-", dim.attrib['id'])



In [12]:
flow_msg

<sdmx.StructureMessage>
  <Header>
    id: 'IDREF652'
    prepared: '2025-05-14T21:53:28.305586+02:00'
    receiver: <Agency Unknown>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (1): CAT_92_1225_86493
  CategoryScheme (1): SEP
  Codelist (38): CL_BASE_YEAR CL_BRANCA_ATTIVITAREV2 CL_CONTAB_NOTE CL_...
  ConceptScheme (2): CONTAB_DOMAIN CROSS_DOMAIN
  DataflowDefinition (1): 92_1225
  DataStructureDefinition (1): DCCN_ANA1

In [13]:
flow_msg.structure 

{'DCCN_ANA1': <DataStructureDefinition IT1:DCCN_ANA1(1.0): National Accounts annual main aggregates>}

In [14]:
dfd.structure 

<DataStructureDefinition IT1:DCCN_ANA1(1.0): National Accounts annual main aggregates>

In [15]:
flow_msg.structure

{'DCCN_ANA1': <DataStructureDefinition IT1:DCCN_ANA1(1.0): National Accounts annual main aggregates>}

In [16]:
dsd = flow_msg.structure 
print(dsd) # Show the data structure definition referred to by the data flow

{'DCCN_ANA1': <DataStructureDefinition IT1:DCCN_ANA1(1.0): National Accounts annual main aggregates>}


In [17]:
flow_msg.codelist.CL_BRANCA_ATTIVITAREV2.items

{'V': <Code V: total economic activities>,
 'VNM': <Code VNM: non market economic activities>,
 'VA': <Code VA: agriculture, forestry and fishing>,
 'V01_02': <Code V01_02: crop and animal production, hunting and related service activities, forestry>,
 'V01': <Code V01: crop and animal production, hunting and related service activities>,
 'V02': <Code V02: forestry and logging>,
 'V03': <Code V03: fishing and aquaculture>,
 'VB_F': <Code VB_F: mining and quarrying, manufacturing, electricity, gas, steam and air conditioning supply, water supply, sewerage, waste management and remediation activities, construction>,
 'VB_E': <Code VB_E: mining and quarrying, manufacturing, electricity, gas, steam and air conditioning supply, water supply, sewerage, waste management and remediation activities>,
 'VBEVCDEVDEVE': <Code VBEVCDEVDEVE: mining and quarrying, manufacture of coke and refined petroleum products, electricity, gas, steam and air conditioning supply, water supply, sewerage, waste man

In [18]:
flow_msg.codelist.CL_BASE_YEAR.items

{'0000': <Code 0000: unspecified>,
 '1980': <Code 1980: year 1980>,
 '1981': <Code 1981: year 1981>,
 '1982': <Code 1982: year 1982>,
 '1989': <Code 1989: year 1989>,
 '1990': <Code 1990: year 1990>,
 '1991': <Code 1991: year 1991>,
 '1992': <Code 1992: year 1992>,
 '1993': <Code 1993: year 1993>,
 '1994': <Code 1994: year 1994>,
 '1995': <Code 1995: year 1995>,
 '1996': <Code 1996: year 1996>,
 '1997': <Code 1997: year 1997>,
 '1998': <Code 1998: year 1998>,
 '1999': <Code 1999: year 1999>,
 '2000': <Code 2000: year 2000>,
 '2001': <Code 2001: year 2001>,
 '2002': <Code 2002: year 2002>,
 '2003': <Code 2003: year 2003>,
 '2004': <Code 2004: year 2004>,
 '2005': <Code 2005: year 2005>,
 '2005_12': <Code 2005_12: December 2005>,
 '2006': <Code 2006: year 2006>,
 '2007': <Code 2007: year 2007>,
 '2008': <Code 2008: year 2008>,
 '2009': <Code 2009: year 2009>,
 '2010': <Code 2010: year 2010>,
 '2011': <Code 2011: year 2011>,
 '2015': <Code 2015: year 2015>}

In [ ]:
istat.data('93_500', key={'REF_AREA': '087015'})

In [58]:
sdmx.to_pandas(data_msg)

NameError: name 'data_msg' is not defined

In [ ]:
import sdmx # pip install sdmx1
istat = sdmx.Client("ISTAT")
exr_flow = exr_msg.dataflow['93_1227']
istat.default_locale = "it"
dsd = exr_flow.structure
dsd.dimensions.components


[<Dimension FREQ>,
 <Dimension REF_AREA>,
 <Dimension DATA_TYPE_AGGR>,
 <Dimension BRKDW_INDUSTRY_NACE_REV2>,
 <Dimension NONFIN_ASSETS>,
 <Dimension EXPEND_PURPOSE_COICOPCOFOG>,
 <Dimension VALUATION>,
 <Dimension ADJUSTMENT>,
 <Dimension PRICE>,
 <Dimension EDITION>,
 <TimeDimension TIME_PERIOD>]

In [26]:
cl = dsd.dimensions.get('BRKDW_INDUSTRY_NACE_REV2').local_representation.enumerated
sdmx.to_pandas(cl)


,name,parent
CL_BRANCA_ATTIVITAREV2,,
V,total economic activities,
VNM,non market economic activities,V
VA,"agriculture, forestry and fishing",V
V01_02,"crop and animal production, hunting and relate...",VA
V01,"crop and animal production, hunting and relate...",VA
...,...,...
U,services provided by extraterritorial organisa...,RTU
G-I_K_L_N_Q-U,services (except information and communication...,GTU
BTDAE36,"industry (except construction, sewerage, waste...",


In [5]:
exr_msg.response.url

'https://esploradati.istat.it/SDMXWS/rest/dataflow/all/93_1227/latest?references=all'

In [6]:
exr_msg

<sdmx.StructureMessage>
  <Header>
    id: 'IDREF9650'
    prepared: '2025-03-10T22:10:34.276475+01:00'
    receiver: <Agency Unknown>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (1): CAT_93_1227_86574
  CategoryScheme (1): DATAWAREHOUSE
  Codelist (37): CL_BASE_YEAR CL_BRANCA_ATTIVITAREV2 CL_CONTAB_NOTE CL_...
  ConceptScheme (2): CONTAB_DOMAIN CROSS_DOMAIN
  DataflowDefinition (1): 93_1227
  DataStructureDefinition (1): DCCN_TNA1

In [30]:
# FREQ, ATECO_2007,ADDETTI,CLASSE_ETA,CLASSE_ETA_IMPR,FORMGIUR,PAESE_NASC,ITTER107,SESSO,TIPO_DATO, TIME_PERIOD

key = dict(
    FREQ="A", 
    ADDETTI = 'TOTAL',
    CLASSE_ETA="TOTAL", 
    ITTER107="ITG1", 
    POSIZPROF= "9", 
    PROFESSIONE2011="99", 
    PROFILO_PROF = "99",
    TEMPO_PIENO_PARZ="9", 
    SESSO="9",
    TITOLO_STUDIO = "99")
params = dict(startPeriod='2023', endPeriod='2023')

data_msg = istat.data('183_203', key=key, params=params)
data = data_msg.data[0]
data_msg.response.url
# len(data.series)

data_df2 = sdmx.to_pandas(
        data,
        datetime={
                'dim': 'TIME_PERIOD',
                'freq': 'FREQ',
                'axis': 1
            }
    )
data_df2

HTTPError: 404 Client Error: Not Found for url: https://esploradati.istat.it/SDMXWS/rest/dataflow/all/183_203/latest?references=all

## richiesta diretta

In [32]:
key = dict(
    FREQ="A", 
    ATECO_2007 = "0010+0011+0012+0020+0025+0026+A+F",
    CARATT_OCC = "9", 
    CITTADINANZA='TOTAL', 
    CLASSE_ETA="Y15-89", 
    ITTER107="ITG1", 
    POSIZPROF= "9", 
    PROFESSIONE2011="99", 
    PROFILO_PROF = "99",
    TEMPO_PIENO_PARZ="9", 
    SESSO="9",
    TITOLO_STUDIO = "99")
params = dict(startPeriod='2023', endPeriod='2023')

data_msg = istat.data('150_938', key=key, params=params)
data = data_msg.data[0]
data_msg.response.url
# len(data.series)

data_df2 = sdmx.to_pandas(
        data,
        datetime={
                'dim': 'TIME_PERIOD',
                'freq': 'FREQ',
                'axis': 1
            }
    )
data_df2


# ATECO_2007="0010+A+0011+0012", 
# IT+ITC1+ITC2+ITC3+ITC4+ITD1+ITD2+ITD3+ITD4+ITD5+ITE1+ITE2+ITE3+ITE4+ITF1+ITF2+ITF3+ITF4+ITF5+ITF6+ITG1+ITG2

data_final = data_df2.xs(
        key=("9","TOTAL","Y15-89","9","99","99","99","9","9","99"),
        level=("CARATT_OCC","CITTADINANZA","CLASSE_ETA","POSIZPROF","PROFESSIONE2001","PROFESSIONE2011","PROFILO_PROF","TEMPO_PIENO_PARZ","SESSO","TITOLO_STUDIO")
    )

print(data_msg.response.url)
data_final


2024-06-24 10:13:56,734 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


https://sdmx.istat.it/SDMXWS/rest/data/150_938/A..0010+0011+0012+0020+0025+0026+A+F.9.TOTAL.Y15-89.ITG1.9..99.99.9.9.99?startPeriod=2023&endPeriod=2023


TIME_PERIOD                      2023
ATECO_2002 ATECO_2007 ITTER107       
0010       0010       ITG1     1410.8
           0011       ITG1      247.4
           0012       ITG1     1042.3
           0020       ITG1      147.7
           0025       ITG1      739.5
           0026       ITG1      302.8
           A          ITG1      121.1
           F          ITG1       99.6

In [25]:
data_df2

TIME_PERIOD                                                                                                                                                     2020  \
ATECO_2002 ATECO_2007 CARATT_OCC CITTADINANZA CLASSE_ETA ITTER107 POSIZPROF PROFESSIONE2001 PROFESSIONE2011 PROFILO_PROF TEMPO_PIENO_PARZ SESSO TITOLO_STUDIO          
0010       0010       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1305.3   
           0011       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             207.0   
           0012       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             986.0   
           0020       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             128.5   

TIME_PERIOD                                                                                                                                                     2021  \
ATECO_2002 ATECO_2007 CARATT_OCC CITTADINANZA CLASSE_ETA ITTER107 POSIZPROF PROFESSIONE2001 PROFESSIONE2011 PROFILO_PROF TEMPO_PIENO_PARZ SESSO TITOLO_STUDIO          
0010       0010       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1310.9   
           0011       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             219.1   
           0012       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             974.3   
           0020       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             123.8   

TIME_PERIOD                                                                                                                                                     2022  \
ATECO_2002 ATECO_2007 CARATT_OCC CITTADINANZA CLASSE_ETA ITTER107 POSIZPROF PROFESSIONE2001 PROFESSIONE2011 PROFILO_PROF TEMPO_PIENO_PARZ SESSO TITOLO_STUDIO          
0010       0010       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1337.0   
           0011       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             223.9   
           0012       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1000.5   
           0020       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             124.1   

TIME_PERIOD                                                                                                                                                     2023  
ATECO_2002 ATECO_2007 CARATT_OCC CITTADINANZA CLASSE_ETA ITTER107 POSIZPROF PROFESSIONE2001 PROFESSIONE2011 PROFILO_PROF TEMPO_PIENO_PARZ SESSO TITOLO_STUDIO         
0010       0010       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1410.8  
           0011       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99             247.4  
           0012       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                9     99            1042.3  
           0020       9          TOTAL        Y15-89     ITG1     9         99              99              99           9                

In [24]:
print(data_final)

TIME_PERIOD           2020   2021   2022   2023
ATECO_2007 ITTER107                            
0010       ITG1     1305.3 1310.9 1337.0 1410.8
0011       ITG1      207.0  219.1  223.9  247.4
0012       ITG1      986.0  974.3 1000.5 1042.3
0020       ITG1      128.5  123.8  124.1  147.7
